In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import MultiLearn_GLRM
from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('./Experiments/ESLR/prostate/prostate.data', delimiter='\t', index_col=0)
df = df.sample(frac=1).reset_index(drop=True)

mask_train = df.pop('train')
df_y = df.pop('lpsa')

X_train = df[mask_train == 'T']
Y_train = df_y[mask_train == 'T']
Y_train = np.array(Y_train).reshape(-1,1)


X_test = df[mask_train != 'T']
Y_test = df_y[mask_train != 'T']
Y_test = np.array(Y_test).reshape(-1,1)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

params = {
    "train_hyper_params": None,
    "test_hyper_params": None,
    "beta": None,
    "embedding_matrix": None,
    "train_error": None,
    "test_error": None
}

In [8]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.semisupervised_params import Multilearn_GLRM_Semisupervised_Train_Params, Multilearn_GLRM_Semisupervised_Test_Params

embedding_dim = 5
n_class = 1

A_prime, X_prime, Y_prime = generate_AXY.get_semisupervised_glrm_train_form(X_train, Y_train, n_class, embedding_dim)
[GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions] = Multilearn_GLRM_Semisupervised_Train_Params(A_prime, X_prime, Y_prime, n_class)

params["train_hyper_params"] = [GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions]

num_iterations=100000
learning_rate=0.01
result = Multi_Learn.alternating_minimization(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions, num_iterations, learning_rate, n_class)


epoch: 0
iter: 386 Total loss: tf.Tensor(4.573163934808955, shape=(), dtype=float64)
iter: 59 Total loss: tf.Tensor(1.7669164500028334, shape=(), dtype=float64)
epoch: 1
iter: 155 Total loss: tf.Tensor(0.9104548625682602, shape=(), dtype=float64)
iter: 10 Total loss: tf.Tensor(0.8389521489138734, shape=(), dtype=float64)
epoch: 2
iter: 55 Total loss: tf.Tensor(0.785934297951629, shape=(), dtype=float64)
iter: 5 Total loss: tf.Tensor(0.7827244209536408, shape=(), dtype=float64)
epoch: 3
iter: 259 Total loss: tf.Tensor(0.7527138291528178, shape=(), dtype=float64)
iter: 116 Total loss: tf.Tensor(0.7514508145929245, shape=(), dtype=float64)
epoch: 4
iter: 367 Total loss: tf.Tensor(0.7495283015307668, shape=(), dtype=float64)
iter: 3 Total loss: tf.Tensor(0.7508748266839278, shape=(), dtype=float64)
epoch: 4 Total loss: tf.Tensor(0.7510585601086077, shape=(), dtype=float64)
Final loss: tf.Tensor(0.7510585601086077, shape=(), dtype=float64) best loss: tf.Tensor(0.7510585601086077, shape=(), 

In [9]:
beta = result[1][1:,0:n_class]
Y_final = result[1][1:embedding_dim+1,n_class:]
params["beta"] = beta
params["embedding_matrix"] = Y_final

res = np.matmul(result[0], result[1][:,0:n_class])
params["train_error"] = np.mean((Y_train - res)**2)
params["train_error"]

0.49761031816784335

In [ ]:
# res = np.matmul(result[0], result[1][:,0:n_class])
# np.mean((Y_train - res)**2)
# 0.4495674496068231

In [ ]:
for i in range(0, Y_train.shape[0]):
    print(Y_train[i], res[i])

In [10]:
A = X_test
A_prime =tf.constant(A)


X = np.random.standard_normal((A.shape[0], embedding_dim))
X_prime = X
X_prime = tf.Variable(X_prime)

Y_prime = Y_final

[GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, X_grad_restrictions, Y_grad_restrictions] =  Multilearn_GLRM_Semisupervised_Test_Params(A_prime, X_prime, Y_prime)
params["test_hyper_params"] = [GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, X_grad_restrictions, Y_grad_restrictions]

num_iterations=50000
learning_rate=0.009
results_log = []
for i in range(0,10):
    A = X_test
    A_prime =tf.constant(A)


    X = np.random.uniform(-1,1, (A.shape[0], embedding_dim))
    X_prime = X
    X_prime = tf.Variable(X_prime)

    Y_prime = Y_final

    result_val = Multi_Learn.predict(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, num_iterations, learning_rate)
    val_pred = np.hstack((result_val[0], A_prime))
    val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
    regress_val = np.matmul(val_pred, beta)
    print(np.mean((Y_test - regress_val)**2))
    results_log.append(np.mean((Y_test - regress_val)**2))

params["test_error"] = results_log

iter: 0 Total loss: tf.Tensor(1.529842555045751, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(0.7898710020225588, shape=(), dtype=float64)
iter: 119 Total loss: tf.Tensor(0.7636602785599883, shape=(), dtype=float64)
0.490090077931291
iter: 0 Total loss: tf.Tensor(1.5327151629238407, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(0.7859905077684408, shape=(), dtype=float64)
iter: 122 Total loss: tf.Tensor(0.759476668879253, shape=(), dtype=float64)
0.49005512554692743
iter: 0 Total loss: tf.Tensor(1.5593074435602872, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(0.7898472921857284, shape=(), dtype=float64)
iter: 123 Total loss: tf.Tensor(0.7609252171727972, shape=(), dtype=float64)
0.4900664107714302
iter: 0 Total loss: tf.Tensor(1.5613645877080988, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(0.7921973851821976, shape=(), dtype=float64)
iter: 128 Total loss: tf.Tensor(0.7578234800024003, shape=(), dtype=float64)
0.49005951285335486
iter: 0 T

In [45]:
import json

.tolist()
with open('./logs/Prostate_0.49_MSE.json', 'w') as f:
    json.dump(params, f)
params

FileNotFoundError: [Errno 2] No such file or directory: './logs/Prostate_0.49_MSE.json'

In [ ]:
"""
beta
<tf.Tensor: shape=(14, 1), dtype=float64, numpy=
array([[-3.38347402e-03],
       [ 6.39743435e-02],
       [ 9.40026586e-02],
       [ 1.21583354e-03],
       [ 4.05847277e-01],
       [ 4.73958481e-01],
       [ 2.62763281e-01],
       [-4.20586971e-02],
       [ 1.75150903e-01],
       [ 1.80748655e-01],
       [-1.46233690e-03],
       [ 2.20528090e-02],
       [ 1.31451859e-01],
       [ 2.43577201e+00]])>

<tf.Tensor: shape=(14, 1), dtype=float64, numpy=
array([[-3.12748490e-01],
       [ 2.25695045e-01],
       [ 2.33751380e-01],
       [-2.30906162e-01],
       [ 9.02321698e-02],
       [ 4.54149462e-01],
       [ 2.19087587e-01],
       [-4.97710150e-02],
       [ 1.59283026e-01],
       [ 1.42314440e-01],
       [-1.58384697e-03],
       [ 2.94253544e-02],
       [ 9.60553143e-02],
       [ 2.44303569e+00]])>
"""

In [ ]:
df.head()

In [ ]:
val_pred = np.hstack((result_val[0], A_prime))
val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
regress_val = np.matmul(val_pred, beta)
np.mean((Y_test - regress_val)**2)

In [ ]:
regress_val

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, predictions_val), confusion_matrix(Y_train, predictions)

In [ ]:
409/(409+200), 409/(409+857)

In [ ]:
305/(305+133), 305/(305+961)

In [ ]:
2027/(2027+538), 2027/(2027+3343)

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train_embedding = result[0][:, 1:embedding_dim+1]
X_train_embedding = X_train_embedding.numpy()

X_test_embedding = result_val[0].numpy()


clf = LogisticRegression().fit(X_train_embedding, Y_train)
pred_logistic = clf.predict(X_train_embedding)
pred_logistic_val = clf.predict(X_test_embedding)
accuracy_score(Y_train, pred_logistic), accuracy_score(Y_test, pred_logistic_val)

In [ ]:
confusion_matrix(Y_test, pred_logistic_val), confusion_matrix(Y_train, pred_logistic)

In [ ]:
328/(328+236), 328/(328+938)

In [ ]:
1565/(1565+650), 1565/(1565+3805)

In [ ]:
beta[5:]

In [ ]:
A_prime[0]

In [ ]:
x_embeddings

In [ ]:
prob_val[0:10]

In [ ]:
prob[0:10]

In [ ]:
temp = tf.Variable([1,1,1])
tf.where(temp!=1).shape[0]
# tf.where([True, False, False, True]).numpy()

In [ ]:
np.min(val_pred,axis=0)

In [ ]:
np.min(result[0],axis=0)

In [ ]:
beta

In [ ]:
result[0][3:10,:]